<a href="https://colab.research.google.com/github/kenny-vb244/DTSC-3020.020.2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [ ]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [17]:
%%writefile q1_crm_cleanup.py
from pathlib import Path
import re

def clean_contacts(input_file="contacts_raw.txt", output_file="contacts_clean.csv"):

  email_pattern = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
  contacts_file = Path(input_file)
  contacts_clean_file = Path(output_file)

  validated_rows = set()
  cleaned_rows = []


  line_pattern = re.compile(r'^(.*?)\s*<?([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})>?\s*,?\s*(.*)$')


  try:
    with contacts_file.open("r", encoding="utf-8") as file_object: # Specify encoding as utf-8
      for line in file_object:
        match = line_pattern.match(line.strip())
        if not match:
            continue

        name, email, phone_part = match.groups()

        name = name.strip().strip('"').strip(',')
        email = email.strip()
        phone = phone_part.strip()


        phone = re.sub(r"\D", "", phone)

        if len(phone) >= 10:
          phone = phone[-10:]
        else:
            phone = ""

        if not re.fullmatch(email_pattern, email):
            continue

        email_key = email.casefold()

        if email_key in validated_rows:
          continue

        validated_rows.add(email_key)

        cleaned_rows.append((name, email, phone))

  except FileNotFoundError:
    print("Sorry, the file does not exist.")
    return

  with contacts_clean_file.open("w", encoding="utf-8") as file_object: # Specify encoding as utf-8
    file_object.write("Name, Email, Phone\n")
    for name, email, phone in cleaned_rows:
      file_object.write(f"{name},{email},{phone}\n")

if __name__ == "__main__":
  clean_contacts()

Overwriting q1_crm_cleanup.py


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [18]:
import unittest
import re
from q1_crm_cleanup import clean_contacts # Import the function to be tested
from pathlib import Path

class TestCRMCleanup(unittest.TestCase):

  def test_email_validation(self):
    input_data = """John Doe,john@example.com,555-123-4567
                    Bad User,bademail@@example,1112223333"""
    Path("test_raw.txt").write_text(input_data)

    clean_contacts("test_raw.txt", "test_clean.csv")
    output_data = Path("test_clean.csv").read_text().strip().split("\n")
    self.assertEqual(len(output_data), 2)
    self.assertIn("john@example.com", output_data[1])

  def test_phone_normalization(self):
    input_data = "Ava Roommate,ava@example.com,(555) 987-6543"
    Path("test_raw.txt").write_text(input_data)

    clean_contacts("test_raw.txt", "test_clean.csv")
    output_data = Path("test_clean.csv").read_text().strip().split("\n")[1]
    self.assertTrue(output_data.endswith("5559876543"))

  def test_parsing(self):
    input_data = '"Alyssa Coworker" <alyssa@work.net>, 444-222-1111'
    Path("test_raw.txt").write_text(input_data)

    clean_contacts("test_raw.txt", "test_clean.csv")

    output_data = Path("test_clean.csv").read_text().strip().split("\n")[1]
    self.assertTrue(output_data.startswith("Alyssa Coworker,alyssa@work.net"))

  def test_deduplication(self):
    input_data = """Emma Bestie, e@example.com,111111111
                    Emma Bestie, e@example.com,222222222"""
    Path("test_raw.txt").write_text(input_data)

    clean_contacts("test_raw.txt", "test_clean.csv")
    output_data = Path("test_clean.csv").read_text().strip().split("\n")
    self.assertEqual(len(output_data), 2)

unittest.main(argv=[''], exit=False, verbosity=2)

test_deduplication (__main__.TestCRMCleanup.test_deduplication) ... ok
test_email_validation (__main__.TestCRMCleanup.test_email_validation) ... ok
test_parsing (__main__.TestCRMCleanup.test_parsing) ... ok
test_phone_normalization (__main__.TestCRMCleanup.test_phone_normalization) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.016s

OK


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
